In [14]:
import math
import sys
import required_functions
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment
inff = 1000000000

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [15]:
def get_current_coords(coords, frame):
    rows = coords.index[coords.FRAME == frame]
    return rows

In [16]:
def assign_new_IDs(coords, point_rows):
    max_ID = coords.TRACK_ID.max()
    for point_row in point_rows:
        if coords.TRACK_ID[point_row] == 0:
            max_ID = max_ID + 1
            coords.TRACK_ID.at[point_row] = max_ID        

In [17]:
def get_all_tracks(coords):    
    unique_IDs = np.unique(coords.TRACK_ID)
    track_rows = []
    for unique_ID in unique_IDs:
        if unique_ID != 0:
            instances = coords.index[coords.TRACK_ID == unique_ID]    
            track_rows.append(instances[-1:][0])
    return track_rows

In [18]:
def get_available_tracks(coords, start_frame, end_frame):
    rows = coords.index[(coords.FRAME >= start_frame) & (coords.FRAME <= end_frame)]
    available_rows = []
    unique_IDs = np.unique(coords.TRACK_ID[rows])
    for unique_ID in unique_IDs:
        if unique_ID != 0:
            instances = coords.index[(coords.TRACK_ID == unique_ID)]    
            available_rows.append(instances[-1:][0])
    return available_rows

In [19]:
def calculate_cost(coords,track_row,point_row,thresh):
    dx = coords.X[point_row] - coords.X[track_row]
    dy = coords.Y[point_row] - coords.Y[track_row]
    d = float(math.sqrt(float(dx*dx + dy*dy)))
    if d > thresh:
        d = inff
    return d

In [20]:
def calculate_cost_matrix(coords,track_rows,point_rows,thresh):
    costs = np.empty((len(track_rows),len(point_rows)))
    for track_i, track_row in enumerate(track_rows):
        for point_i,point_row in enumerate(point_rows):            
            cost = calculate_cost(coords,track_row,point_row,thresh)
            costs[track_i,point_i] = cost
    return costs

In [21]:
def assign_IDs(coords, track_rows, point_rows, costs):
    assignments = linear_sum_assignment(costs)
    for track_assignment, point_assignment in zip(assignments[0],assignments[1]):
        if costs[track_assignment,point_assignment] < inff:
            track_row = track_rows[track_assignment]
            point_row = point_rows[point_assignment]
            ID = coords.TRACK_ID[track_row]
            coords.TRACK_ID.at[point_row] = ID

In [22]:
%matplotlib widget

np.set_printoptions(precision=3,threshold=sys.maxsize)
linking_thresh = 5
frame_thresh = 3
path = "./images/01/"
images = required_functions.load_images(path);
path = "./images/coordinates/experiment_1.csv"
coords = required_functions.load_coordinates(path);
rows_0 = get_current_coords(coords,0)
assign_new_IDs(coords,rows_0)
n_frames = images.shape[2]
for frame in range (1,n_frames):    
    sys.stdout.write("\rProcessing frame %i" % frame)
    start_frame = frame - frame_thresh
    end_frame = frame-1
    track_rows = get_available_tracks(coords,start_frame,end_frame)
    point_rows = get_current_coords(coords,frame)
    thresh = 10
    costs = calculate_cost_matrix(coords,track_rows,point_rows,thresh)
    assign_IDs(coords, track_rows, point_rows, costs)
    assign_new_IDs(coords,point_rows)

Loading images from " ./images/01/ "

Reading image 92 of 92
Loaded image shape:  (256, 256, 92)
Loading coordinates from " ./images/coordinates/experiment_1.csv "
Loaded data shape:  (2008, 6)
 
Processing frame 91

In [23]:
required_functions.show_overlay(images,coords,True)

Rendering frame 92 of 92

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='frame', max=91), Output()), _dom_classes=('widget-intera…

In [24]:
print(coords)

        inde      ID           X           Y  FRAME  TRACK_ID
0        0.0     1.0  142.210522   10.880823    0.0       1.0
1        1.0     2.0   48.384861   26.279000    0.0       2.0
2        2.0     3.0  206.925981   45.645939    0.0       3.0
3        3.0     4.0   19.775794   50.961693    0.0       4.0
4        4.0     5.0   99.646064   76.501039    0.0       5.0
...      ...     ...         ...         ...    ...       ...
2003  2003.0  2004.0  177.924070  181.070166   91.0      17.0
2004  2004.0  2005.0  114.266401  196.790956   91.0      97.0
2005  2005.0  2006.0  107.717568  198.645946   91.0     121.0
2006  2006.0  2007.0  185.106294  238.986352   91.0      61.0
2007  2007.0  2008.0  190.770833  238.873698   91.0     134.0

[2008 rows x 6 columns]


In [25]:
x = coords['X']
y = coords['Y']
t = coords['FRAME']

In [26]:
fig = plt.figure(figsize=(200,200))
axes = plt.axes(projection='3d')
axes.set_xlabel("x")
axes.set_ylabel("y")
axes.set_zlabel("time")
axes.scatter(x,y,t,s=2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …